In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
from iris_plant_visualizer import IrisPlantVisualizer
from iris_plant_animation_visualizer import IrisPlantVisualizer as viz2

import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import visualization_utils as viz_utils


In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Rgba
import time
import pydrake.multibody.rational as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptions, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions

In [4]:
from pydrake.geometry.optimization_dev import (CspaceFreePolytope, 
                                               SeparatingPlaneOrder)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

We first set up a simple 2-DOF system and visualize both the plant and the configuration constraint.Click on the two links at the bottom to view the plant and the configuration space.

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
#construct our robot
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))
locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()
idx = 0
q0 = [0.0, 0.0]
val = 1.7
q_low  = np.array([-val, -val])
q_high = np.array([val, val])
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

# The object we will use to perform our certification.
cspace_free_polytope = CspaceFreePolytope(plant, scene_graph, SeparatingPlaneOrder.kAffine, q_star)
cspace_frame = RigidTransform(RotationMatrix.MakeYRotation(np.pi/2)@RotationMatrix.MakeZRotation(np.pi/2), np.array([0,3,1]))
# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = viz2(plant, builder, scene_graph, cspace_free_polytope, cspace_frame, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.2, 
                                          num_points = 100)
# visualizer.meshcat.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [6]:
visualizer.meshcat.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

## Set up the sliders so we can move the plant around manually

You can use the sliders below to move the two degrees of freedom of the plant around. A green dot will appear in the TC-space visualization describing the current TC-space configuration.

In [7]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0', step = 0.02))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1', step = 0.02))

q = np.array([1.70, 0])
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7, step=0.02)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7, step=0.02)

# Generate and Certify Regions

Around some nominal seed postures, we will grow certified regions by seeding our alternation algorithm using a small initial polytope.

In [8]:
# Some seedpoints
seed_points_q = np.array([   
                              [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.5,-1.08],
                              [-1.06, 0.48]
                              ])

seed_points = np.array([Ratfk.ComputeSValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
start = seed_points[0]
end = seed_points[-1]

visualizer.plot_cspace_points(seed_points, "/iris_seed_points", radius = 0.02)
    


default_scale = 1e-2
L1_ball = HPolyhedron.MakeL1Ball(2)
Linf_ball = HPolyhedron.MakeBox(-np.ones(2), np.ones(2))

template_C = np.vstack([L1_ball.A(), Linf_ball.A()])
template_d = np.hstack([default_scale*L1_ball.b(), default_scale/np.sqrt(2)*Linf_ball.b()])


def make_default_polytope_at_point(seed_point):
    return HPolyhedron(template_C, template_d + template_C @ seed_point)


# colors to plot the region. Chosen for color-blind compatibility
default_alpha = 0.2
colors_dict = {
    0: Rgba(0.565, 0.565, 0.565, default_alpha), # gray
    1: Rgba(0.118, 0.533, 0.898, default_alpha), # bluish
    2: Rgba(1,     0.757, 0.027, default_alpha), # gold
    3: Rgba(0,     0.549, 0.024, default_alpha), # green   
    4: Rgba(0.055, 0.914, 0.929, default_alpha), # teal 
}

initial_regions = [(make_default_polytope_at_point(s), colors_dict[i]) for i, s in enumerate(seed_points)]
# visualizer.add_group_of_regions_to_visualization(initial_regions, "initial_regions",
#                             wireframe = False,
#                            opacity = default_alpha)
visualizer.show_res_q(q_star)

visualizer.add_plane_indices_of_interest(118)

In [9]:
visualizer._plane_indices_of_interest

[118]

## First we set up some options for our different certification modes

In [10]:
# set up the certifier and the options for different search techniques
solver_options = SolverOptions()
# set this to 1 if you would like to see the solver output in terminal.
solver_options.SetOption(CommonSolverOption.kPrintToConsole, 0)

# The options for when we search for new planes and positivity certificates given the polytopes
find_separation_certificate_given_polytope_options = CspaceFreePolytope.FindSeparationCertificateGivenPolytopeOptions()
find_separation_certificate_given_polytope_options.num_threads = -1
find_separation_certificate_given_polytope_options.verbose = False
find_separation_certificate_given_polytope_options.solver_options = solver_options
find_separation_certificate_given_polytope_options.ignore_redundant_C = False

# The options for when we search for a new polytope given positivity certificates.
find_polytope_given_lagrangian_option = CspaceFreePolytope.FindPolytopeGivenLagrangianOptions()
find_polytope_given_lagrangian_option.solver_options = solver_options
find_polytope_given_lagrangian_option.ellipsoid_margin_cost = CspaceFreePolytope.EllipsoidMarginCost.kGeometricMean
find_polytope_given_lagrangian_option.search_s_bounds_lagrangians = True
find_polytope_given_lagrangian_option.ellipsoid_margin_epsilon = 1e-8


bilinear_alternation_options = CspaceFreePolytope.BilinearAlternationOptions()
bilinear_alternation_options.max_iter = 50
bilinear_alternation_options.convergence_tol = 1e-5
bilinear_alternation_options.find_polytope_options = find_polytope_given_lagrangian_option
bilinear_alternation_options.find_lagrangian_options = find_separation_certificate_given_polytope_options

binary_search_options = CspaceFreePolytope.BinarySearchOptions()
binary_search_options.find_lagrangian_options = find_separation_certificate_given_polytope_options
binary_search_options.scale_min = 1
binary_search_options.scale_max = 50
binary_search_options.max_iter = 100

Clim = np.vstack([np.eye(q_star.shape[0]), -np.eye(q_star.shape[0])])
dlim = np.concatenate([Ratfk.ComputeSValue(q_high, q_star), -Ratfk.ComputeSValue(q_low, q_star)])
def polytope_with_joint_limits(C, d):
    return HPolyhedron(np.vstack([C, Clim]), np.concatenate([d, dlim]))

## Seeding and Certifying with a Stronger Heuristic
We have also implemented another, strong heuristic for proposing good initial regions based on non-linear optimization. See Appendix TODO of our paper TODO for details

In [11]:
iris_regions = []
iris_ellipses = []

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.configuration_space_margin = 1e-3
iris_options.relative_termination_threshold = 0.001

context_for_iris = visualizer.diagram_context
for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, q_star)
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context_for_iris), q)
    r = IrisInRationalConfigurationSpace(plant, 
                                         plant.GetMyContextFromRoot(context_for_iris),
                                         q_star, iris_options)
    iris_regions.append((r, colors_dict[i]))
    iris_ellipses.append(r.MaximumVolumeInscribedEllipsoid())
    

# visualizer.add_group_of_regions_to_visualization(iris_regions, "/uncertified_iris", 
#                                                             wireframe = False, opacity = 0.2)

In [12]:
# binary_search_options_for_iris = CspaceFreePolytope.BinarySearchOptions()
# binary_search_options_for_iris.scale_max = 1.0 # it is highly unlikely that the regions can grow, but we can try
# binary_search_options_for_iris.find_lagrangian_options = find_separation_certificate_given_polytope_options
# binary_search_options_for_iris.max_iter = 50
# binary_search_region_certificates_for_iris = dict.fromkeys([tuple(s) for s in seed_points])
# for i, (s, (initial_region, color)) in enumerate(zip(seed_points, iris_regions)):
#     print(f"starting seedpoint {i+1}/{len(iris_regions)}")
#     time.sleep(0.2)    
#     cert = cspace_free_polytope.BinarySearch(set(),
#                                                     initial_region.A(),
#                                                     initial_region.b(), 
#                                                     initial_region.MaximumVolumeInscribedEllipsoid().center(), 
#                                                     binary_search_options_for_iris)
#     binary_search_region_certificates_for_iris[tuple(s)] = [(polytope_with_joint_limits(cert.C, cert.d),
#                                                              cert, color)]
# for i, result in enumerate(binary_search_region_certificates_for_iris.values()):
#     group_name = f"/certified-iris-bin_seed_point_{i}"
#     visualizer.add_group_of_regions_and_certs_to_visualization(result, group_name, 
#                                                         wireframe = False, opacity = 0.2)
# #     tmp = [(r[0], r[2]) for r in result]
# #     visualizer.add_group_of_regions_to_visualization(tmp, group_name, 
# #                                                             wireframe = False, opacity = 0.2)

# visualizer.show_res_q(q)

In [13]:
from time import perf_counter
bilinear_alternation_results_by_seed_point = dict.fromkeys([tuple(s) for s in seed_points])
times_per_region = []
for i, (s, (initial_region, color)) in enumerate(zip(seed_points, initial_regions)):
    print(f"starting seedpoint {i+1}/{len(initial_regions)}")
    time.sleep(0.1) # make the above print before the Drake log.
    bilinear_alternation_options.find_polytope_options.s_inner_pts = s
    t_start = time.perf_counter()
    result = cspace_free_polytope.SearchWithBilinearAlternation(set(), initial_region.A(),
                                                                initial_region.b(),
                                                                bilinear_alternation_options)
    t_end = time.perf_counter()
    times_per_region.append(t_end - t_start)
    bilinear_alternation_results_by_seed_point[tuple(s)] = [(polytope_with_joint_limits(cert.C, cert.d),
                                                             cert, color) for cert in result]
    

INFO:drake:det(Q) at the beginning is 4.900500382473016e-05


starting seedpoint 1/5


INFO:drake:Iteration 0: det(Q)=0.00014576238925864208
INFO:drake:Iteration 1: det(Q)=0.0002806159113297737
INFO:drake:Iteration 2: det(Q)=0.0005344000732312085
INFO:drake:Iteration 3: det(Q)=0.0009169467132802288
INFO:drake:Iteration 4: det(Q)=0.0013848549945793937
INFO:drake:Iteration 5: det(Q)=0.0019126336178197192
INFO:drake:Iteration 6: det(Q)=0.00255550225428644
INFO:drake:Iteration 7: det(Q)=0.003337093699566204
INFO:drake:Iteration 8: det(Q)=0.00430669080783459
INFO:drake:Iteration 9: det(Q)=0.005568220382883087
INFO:drake:Iteration 10: det(Q)=0.007114884752347183
INFO:drake:Iteration 11: det(Q)=0.008935629008910621
INFO:drake:Iteration 12: det(Q)=0.011160505913197317
INFO:drake:Iteration 13: det(Q)=0.013856731449179946
INFO:drake:Iteration 14: det(Q)=0.016859548786580456
INFO:drake:Iteration 15: det(Q)=0.020610878066453044
INFO:drake:Iteration 16: det(Q)=0.02530337015678481
INFO:drake:Iteration 17: det(Q)=0.02861864786435216
INFO:drake:Iteration 18: det(Q)=0.031073885939933994


starting seedpoint 2/5


INFO:drake:Iteration 1: det(Q)=0.00036805961026473594
INFO:drake:Iteration 2: det(Q)=0.0007327701038891825
INFO:drake:Iteration 3: det(Q)=0.001456733282687195
INFO:drake:Iteration 4: det(Q)=0.002961332348494822
INFO:drake:Iteration 5: det(Q)=0.005965834368885852
INFO:drake:Iteration 6: det(Q)=0.009942655734023328
INFO:drake:Iteration 7: det(Q)=0.01235925129642312
INFO:drake:Iteration 8: det(Q)=0.013822141909334801
INFO:drake:Iteration 9: det(Q)=0.015100253158656518
INFO:drake:Iteration 10: det(Q)=0.01629489374179905
INFO:drake:Iteration 11: det(Q)=0.01721702132606201
INFO:drake:Iteration 12: det(Q)=0.018057449774436303
INFO:drake:Iteration 13: det(Q)=0.018789068669064324
INFO:drake:Iteration 14: det(Q)=0.019457449285388582
INFO:drake:Iteration 15: det(Q)=0.019972553730983292
INFO:drake:Iteration 16: det(Q)=0.020319434371489833
INFO:drake:Iteration 17: det(Q)=0.0206264546237852
INFO:drake:Iteration 18: det(Q)=0.020905526613063476
INFO:drake:Iteration 19: det(Q)=0.021140206492264196
INFO

...

INFO:drake:Iteration 0: det(Q)=0.0002893803632078695
INFO:drake:Iteration 1: det(Q)=0.000629240698708994
INFO:drake:Iteration 2: det(Q)=0.0010005786406084907
INFO:drake:Iteration 3: det(Q)=0.0015026235141849568
INFO:drake:Iteration 4: det(Q)=0.0022000574340110984
INFO:drake:Iteration 5: det(Q)=0.0030720890177413714
INFO:drake:Iteration 6: det(Q)=0.004162115304839216
INFO:drake:Iteration 7: det(Q)=0.005547234537234709
INFO:drake:Iteration 8: det(Q)=0.0074292604396001775
INFO:drake:Iteration 9: det(Q)=0.010085315083395479
INFO:drake:Iteration 10: det(Q)=0.013281577185173903
INFO:drake:Iteration 11: det(Q)=0.01694537902529072
INFO:drake:Iteration 12: det(Q)=0.02153020666681398
INFO:drake:Iteration 13: det(Q)=0.027388154267308482
INFO:drake:Iteration 14: det(Q)=0.035545598544039415
INFO:drake:Iteration 15: det(Q)=0.04730253089744523
INFO:drake:Iteration 16: det(Q)=0.06925458037467991
INFO:drake:Iteration 17: det(Q)=0.0933919784520877
INFO:drake:Iteration 18: det(Q)=0.10877284559474545
INFO

starting seedpoint 4/5


INFO:drake:Iteration 1: det(Q)=0.00018579145628959672
INFO:drake:Iteration 2: det(Q)=0.0002769318578043369
INFO:drake:Iteration 3: det(Q)=0.0004675590586949072
INFO:drake:Iteration 4: det(Q)=0.0007918939209971374
INFO:drake:Iteration 5: det(Q)=0.0011657905502829118
INFO:drake:Iteration 6: det(Q)=0.0017386046717462675
INFO:drake:Iteration 7: det(Q)=0.002541565057009815
INFO:drake:Iteration 8: det(Q)=0.0034983439448962165
INFO:drake:Iteration 9: det(Q)=0.0046491207103816845
INFO:drake:Iteration 10: det(Q)=0.006070204334967574
INFO:drake:Iteration 11: det(Q)=0.007643903525013378
INFO:drake:Iteration 12: det(Q)=0.009792904967279723
INFO:drake:Iteration 13: det(Q)=0.01334431554136183
INFO:drake:Iteration 14: det(Q)=0.018856065597072914
INFO:drake:Iteration 15: det(Q)=0.02782325429042248
INFO:drake:Iteration 16: det(Q)=0.044361018913048784
INFO:drake:Iteration 17: det(Q)=0.057292038884133555
INFO:drake:Iteration 18: det(Q)=0.06822090668193889
INFO:drake:Iteration 19: det(Q)=0.080595122223269

starting seedpoint 5/5


INFO:drake:Iteration 1: det(Q)=0.00015239078399186516
INFO:drake:Iteration 2: det(Q)=0.00023425033830250453
INFO:drake:Iteration 3: det(Q)=0.0003362913456625858
INFO:drake:Iteration 4: det(Q)=0.00047180029306225185
INFO:drake:Iteration 5: det(Q)=0.000631131513203011
INFO:drake:Iteration 6: det(Q)=0.0008448350871849316
INFO:drake:Iteration 7: det(Q)=0.0011059142547443187
INFO:drake:Iteration 8: det(Q)=0.0014591068950592092
INFO:drake:Iteration 9: det(Q)=0.001984403557036214
INFO:drake:Iteration 10: det(Q)=0.0027109588797051883
INFO:drake:Iteration 11: det(Q)=0.003299000202382135
INFO:drake:Iteration 12: det(Q)=0.004479237882434005
INFO:drake:Iteration 13: det(Q)=0.005494824570979182
INFO:drake:Iteration 14: det(Q)=0.007830253356251222
INFO:drake:Iteration 15: det(Q)=0.011396510162193785
INFO:drake:Iteration 16: det(Q)=0.017585510310220712
INFO:drake:Iteration 17: det(Q)=0.024454094550338586
INFO:drake:Iteration 18: det(Q)=0.032943287186086134
INFO:drake:Iteration 19: det(Q)=0.0381002236

In [14]:
bilinear_alternation_final_results = [rez[-1] for rez in bilinear_alternation_results_by_seed_point.values()]

In [15]:
bilinear_alternation_final_results

[(<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fcfb0>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147f55b0>,
  Rgba(r=0.118, g=0.533, b=0.898, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147f6eb0>,
  Rgba(r=1.0, g=0.757, b=0.027, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fa070>,
  Rgba(r=0.0, g=0.549, b=0.024, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fef30>,
  Rgba(r=0.055, g=0.914, b=0.929, a=0.2))]

In [16]:
for i, result in enumerate(bilinear_alternation_final_results):
    group_name = f"/certified-bil_alt_{i}"
    visualizer.add_group_of_regions_and_certs_to_visualization([result], group_name, 
                                                        wireframe = False, opacity = 0.2)
#     tmp = [(r[0], r[2]) for r in result]
#     visualizer.add_group_of_regions_to_visualization(tmp, group_name, 
#                                                             wireframe = False, opacity = 0.2)

In [17]:
list(bilinear_alternation_results_by_seed_point.values())[0]

[(<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fc370>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f9417d9ec70>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fc3f0>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fc430>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fc470>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f9417cd02b0>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fc4f0>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fc570>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimization.HPolyhedron at 0x7f94147fc5b0>,
  Rgba(r=0.565, g=0.565, b=0.565, a=0.2)),
 (<pydrake.geometry.optimiza

In [18]:
final_regions_with_certs = [v[-1] for v in bilinear_alternation_results_by_seed_point.values()]
final_regions = [r for (r,_, _) in final_regions_with_certs]

In [19]:
from gcs.bezier import BezierGCS
from gcs.linear import LinearGCS
from pydrake.solvers.mosek import MosekSolver
from pydrake.all import PiecewisePolynomial
from visualization_utils import draw_traj

start = seed_points[0]
end = seed_points[-1]
regions = final_regions#[r for (r,_) in iris_regions]

l_gcs = LinearGCS(regions)
l_gcs.addSourceTarget(start, end)
l_gcs.setSolver(MosekSolver())
waypoints_forward = l_gcs.SolvePath(True)[0]
t_final = 5
t_knots_forward = np.linspace(0, t_final,  waypoints_forward.shape[1])
dt = t_knots_forward[1] - t_knots_forward[0]
t_knots = np.concatenate([t_knots_forward, t_knots_forward+t_final+dt])
waypoints = np.hstack([waypoints_forward, waypoints_forward[:, ::-1]])

# t_knots = t_knots_forward
# waypoints = waypoints_forward

lin_traj = PiecewisePolynomial.FirstOrderHold(t_knots, waypoints)

draw_traj(visualizer.meshcat, lin_traj, 100, transform = visualizer.cspace_frame)


In [20]:
t_knots

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [21]:
# def record_animation(animation, num_frames, time_step):
#     for i in range(num_frames):
#         animation.SetProperty(0, f"/frame_{i}", "visible", False)
#         animation.SetProperty(0, f"/s", "visible", False)
#     for i in range(num_frames):
#         frame = animation.frame(i*time_step)
#         if i > 0:
#             animation.SetProperty(i, f"/frame_{i-1}", "visible", False)
# #             animation.SetProperty(i, f"/frame_{i-1}/s", "visible", False)
#         animation.SetProperty(i, f"/frame_{i}", "visible", True)
# #             animation.SetProperty(i, f"/frame_{i}/s", "visible", True)

def record_animation(animation, num_frames, time_points):
    for i in range(num_frames+1):
        animation.SetProperty(0, f"/frame_{i}", "visible", False)
        animation.SetProperty(0, f"/s", "visible", False)
    for i in range(num_frames):
        frame = animation.frame(time_points[i])
        if i > 0:
            animation.SetProperty(frame, f"/frame_{i-1}", "visible", False)
        animation.SetProperty(frame, f"/frame_{i}", "visible", True)
    for i in range(num_frames):
        animation.SetProperty(num_frames-1, f"/frame_{num_frames-1}", "visible", False)
        animation.SetProperty(num_frames-1, f"/s", "visible", False)


In [22]:
def animate_traj_s(self, traj, steps, runtime):
    # loop
    idx = 0
    going_fwd = True
    time_points = np.linspace(0, traj.end_time(), steps)
    frame_count = 0
    for _ in range(runtime):
        t = time_points[idx]
        s = traj.value(t)
        
        self.diagram_context.SetTime(t)
        self.diagram.ForcedPublish(self.diagram_context)
        self.show_res_s(s, frame=frame_count)
        frame_count += 1
        if going_fwd:
            if idx + 1 < steps:
                idx += 1
            else:
                going_fwd = False
                idx -= 1
        else:
            if idx - 1 >= 0:
                idx -= 1
            else:
                going_fwd = True
                idx += 1
    return frame_count



In [23]:
substeps = 100
runtime = substeps
# visualizer.animate_traj_s(lin_traj, substeps, runtime)
visualizer.visualizer.StartRecording()
time_step = 0.01
# frame_count = visualizer.animate_traj_s(lin_traj, substeps, runtime, time_step = time_step)
frame_count = animate_traj_s(visualizer, lin_traj, substeps, runtime)
visualizer.visualizer.PublishRecording()


In [24]:
from pydrake.all import MeshcatAnimation 
time_points = np.linspace(0, lin_traj.end_time(), substeps)
animation = visualizer.visualizer.get_mutable_recording()
record_animation(animation,frame_count,time_points)
# animation.set_loop_mode(MeshcatAnimation.LoopMode.kLoopPingPong)
# animation.set_repetitions(1)
visualizer.visualizer.PublishRecording()

In [25]:
with open("pinball_trajectory.html", "w") as f:
    f.write(visualizer.meshcat.StaticHtml())
    